In [3]:
%cd ~/textureSynthesis-stylegan2-pytorch
import math
import random
import os
import numpy as np

import torch
from torch import nn, autograd, optim
from torch.nn import functional as F
from torch.utils import data
import torch.distributed as dist
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
# from args import args

from model import ModifiedMultiScaleTextureGenerator

/home/pappas/textureSynthesis-stylegan2-pytorch


In [4]:
device = "cuda:0"
np.random.seed(0)
torch.manual_seed(0)

generator = ModifiedMultiScaleTextureGenerator(size=256, style_dim=512, n_mlp=8, channel_multiplier=2, max_texton_size=64, n_textons=1000)
generator.to(device)
generator.eval()
print(generator)

ModifiedMultiScaleTextureGenerator(
  (style): Sequential(
    (0): PixelNorm()
    (1): EqualLinear(512, 512)
    (2): EqualLinear(512, 512)
    (3): EqualLinear(512, 512)
    (4): EqualLinear(512, 512)
    (5): EqualLinear(512, 512)
    (6): EqualLinear(512, 512)
    (7): EqualLinear(512, 512)
    (8): EqualLinear(512, 512)
  )
  (input): BagOfTextonsVariableSize()
  (conv1): ModifiedStyledConv(
    (conv): ModulatedConv2d(512, 512, 3, upsample=False, downsample=False)
    (noise): NoiseInjection()
    (activate): FusedLeakyReLU()
    (texton): BagOfTextonsVariableSize()
  )
  (to_rgb1): ToRGB(
    (conv): ModulatedConv2d(512, 3, 1, upsample=False, downsample=False)
  )
  (convs): ModuleList(
    (0): ModifiedStyledConv(
      (conv): ModulatedConv2d(512, 512, 3, upsample=True, downsample=False)
      (noise): NoiseInjection()
      (activate): FusedLeakyReLU()
      (texton): BagOfTextonsVariableSize()
    )
    (1): ModifiedStyledConv(
      (conv): ModulatedConv2d(512, 512, 3, ups

In [5]:
import torchsummary 

In [8]:
torchsummary.summary(generator,(1,512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         PixelNorm-1                  [-1, 512]               0
       EqualLinear-2                  [-1, 512]         262,656
       EqualLinear-3                  [-1, 512]         262,656
       EqualLinear-4                  [-1, 512]         262,656
       EqualLinear-5                  [-1, 512]         262,656
       EqualLinear-6                  [-1, 512]         262,656
       EqualLinear-7                  [-1, 512]         262,656
       EqualLinear-8                  [-1, 512]         262,656
       EqualLinear-9                  [-1, 512]         262,656
        PixelNorm-10                  [-1, 512]               0
      EqualLinear-11                  [-1, 512]         262,656
      EqualLinear-12                  [-1, 512]         262,656
      EqualLinear-13                  [-1, 512]         262,656
      EqualLinear-14                  [